*version 190910.0*

# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment.

In addition, I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

## Notes

1. Partial marks may be awarded if the answer is incorrect, or removed even if the answer is correct! Within each function show your understanding of course concepts such as regex, `groupby()`, `apply()`, etc. as appropriate to the question. If you unnecessarily hand code data in your functions instead of using pandas cleaning mechanisms you may be docked grades.
2. Do not including data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
3. I highly suggest that you first tackle the correlation questions, as they are all similar and worth the majority of grades for this assignment. This is by design!
4. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
5. This assignment **is not autograded**. Go to the course shell and upload your .ipynb file there as assignment4.ipynb.
6. You can earn up to 100% on this assignment by creating correlations for a single year (2016) for which there is population data. You can earn up to 105% on this assignment by going further, and creating correlations for multiple years (e.g. across the years of data we have on the sports), but it's expected that there would be a short discussion of the limitations of your approach as far as interpretation goes. Finally, if you find new population data that covers a broader set of years and integrate it into a solution across these years and sports and provide that interpretation you can earn up to 110% on the assignment.

# Statistical Assumptions:
### 1: For the variables utilized in each test, assume they are normally distributed.
### 2: For each test, assume the relationship between the two variables is linear.
### 3: For each test, assume all observations are independent of each other.
### 4: For the t-test, don't necessarily need to verify homogeneity of variance since it is a paired t-test.


# Please run the next eight cells before calling each function. Thank you!


In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)


##Question 1 clean NHL dataframe and create mapping dictinary for cites_nhl df. Use only year 2017
nhl_df=pd.read_csv("assets/nhl.csv")
base_df_nhl = nhl_df
base_df_nhl = base_df_nhl.where(base_df_nhl['year']==2017).dropna(subset= ['year'])
base_df_nhl.drop(base_df_nhl.index[34:], inplace = True)
base_df_nhl = base_df_nhl[~base_df_nhl['team'].isin(['Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division'])]
base_df_nhl['team'] = base_df_nhl['team'].str.replace("*","")
base_df_nhl['W'] = base_df_nhl['W'].astype('int64')
base_df_nhl['L'] = base_df_nhl['L'].astype('int64')
base_df_nhl['Win_Loss_Ratio'] = base_df_nhl['W']/(base_df_nhl['W'] + base_df_nhl['L'] ) 
base_df_nhl.set_index('team', inplace = True)
cols = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
base_df_nhl.drop(base_df_nhl.columns[cols],axis=1,inplace=True)
win_loss_dict = base_df_nhl.groupby('team').apply(lambda x: x.Win_Loss_Ratio.tolist()).to_dict()



#####get average win/loss for New York & Los Angeles since multiple teams
base_df_nhl_filtered_newyork = base_df_nhl.reset_index()
base_df_nhl_filtered_newyork =  base_df_nhl_filtered_newyork[(base_df_nhl_filtered_newyork['team']=='New York Rangers') | (base_df_nhl_filtered_newyork['team']== 'New York Islanders') | (base_df_nhl_filtered_newyork['team'] == 'New Jersey Devils')]
average_newyork = base_df_nhl_filtered_newyork['Win_Loss_Ratio'].mean()

base_df_nhl_filtered_LA = base_df_nhl.reset_index()
base_df_nhl_filtered_LA =  base_df_nhl_filtered_LA[(base_df_nhl_filtered_LA['team']=='Los Angeles Kings') | (base_df_nhl_filtered_LA['team']== 'Anaheim Ducks') ]
average_LA = base_df_nhl_filtered_LA['Win_Loss_Ratio'].mean()


In [8]:
#####Question 1 clean cities dataframe NHL and add win/loss ratio using mapping dictionary
cities_nhl=pd.read_html("assets/wikipedia_data.html",  na_values = '—')[1]
cities_nhl=cities_nhl.iloc[:-1,[0,3,5,6,7,8]]
cities_nhl['NHL'] = cities_nhl['NHL'].str.replace(r"\[.*\]","")



#####Only includes metropolitan areas that did not have NA values for NHL column, taken directly from html file.
#####Remove Vegas Golden Knights from final dataframe since W/L value is null and thus has no bearing on question of interest.
#####Did not add keys for teams in metro areas with multiple teams in order to make mapping more intuitive. Still averaging values of all teams and inserting in relevant row on later line.
NHLMetropolitianAreaDict  = {'New York City':'New York Rangers', 
                             'Los Angeles':'Los Angeles Kings', 
                             'San Francisco Bay Area':'San Jose Sharks', 
                             'Chicago':'Chicago Blackhawks', 
                             'Dallas–Fort Worth':'Dallas Stars', 
                             'Washington, D.C.':'Washington Capitals', 
                             'Philadelphia':'Philadelphia Flyers', 
                             'Boston':'Boston Bruins',
                             'Minneapolis–Saint Paul':'Minnesota Wild', 
                             'Denver':'Colorado Avalanche', 
                             'Miami–Fort Lauderdale':'Florida Panthers', 
                             'Phoenix':'Arizona Coyotes', 
                             'Detroit':'Detroit Red Wings',
                             'Toronto':'Toronto Maple Leafs', 
                             'Tampa Bay Area':'Tampa Bay Lightning',
                             'Pittsburgh':'Pittsburgh Penguins',
                             'St. Louis':'St. Louis Blues',
                             'Nashville':'Nashville Predators',
                             'Buffalo':'Buffalo Sabres',
                             'Montreal':'Montreal Canadiens',
                             'Vancouver':'Vancouver Canucks',
                             'Columbus':'Columbus Blue Jackets',
                             'Calgary':'Calgary Flames',
                             'Ottawa':'Ottawa Senators',
                             'Edmonton':'Edmonton Oilers',
                             'Winnipeg':'Winnipeg Jets',
                             'Las Vegas':'Vegas Golden Knights',
                             'Raleigh':'Carolina Hurricanes'}



cities_nhl = cities_nhl.drop(columns=['NFL', 'MLB', 'NBA'])
cities_nhl = cities_nhl.dropna(subset = ['NHL'])
cities_nhl = cities_nhl.drop(cities_nhl[cities_nhl['NHL']== ""].index)
cities_nhl.set_index('Metropolitan area', inplace = True)
for i in range(len(cities_nhl)):
        city = cities_nhl.iloc[i].name
        cities_nhl.set_value(city, 'team', NHLMetropolitianAreaDict[city])
cities_nhl['Win_Loss_Ratio'] = cities_nhl.team.map(win_loss_dict)
cities_nhl['Win_Loss_Ratio'] = cities_nhl['Win_Loss_Ratio'].astype(str)
cities_nhl['Win_Loss_Ratio'] = cities_nhl['Win_Loss_Ratio'].str.replace('[', '')
cities_nhl['Win_Loss_Ratio'] = cities_nhl['Win_Loss_Ratio'].str.replace(']', '')
cities_nhl['Win_Loss_Ratio'] = cities_nhl['Win_Loss_Ratio'].astype(float)
cities_nhl['Population (2016 est.)[8]'] = cities_nhl['Population (2016 est.)[8]'].astype(int)
cities_nhl = cities_nhl.dropna(subset=['Win_Loss_Ratio'])

#####Now, replace New York and LA Win/Loss with average for all teams. Pass into New York and LA Rows
cities_nhl.iat[0,3] = average_newyork
cities_nhl.iat[1,3] = average_LA


#####Lastly, create lists of metropolitan area Pop and Win/Loss Ratio for NHL. Converted to float/int beforehand.
#####Note: 2016 population data not available for NHL, have to use 2016
a_list_nhl = cities_nhl['Population (2016 est.)[8]'].tolist()
b_list_nhl = cities_nhl['Win_Loss_Ratio'].tolist()





In [9]:

##Question 2 clean NBA dataframe and create mapping dictinary for cites_nba df. Use only year 2016
nba_df=pd.read_csv("assets/nba.csv")
base_df_nba = nba_df
base_df_nba = base_df_nba.where(base_df_nba['year']==2016).dropna(subset= ['year'])
base_df_nba = base_df_nba[~base_df_nba['team'].isin(['Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division'])]
base_df_nba['team'] = base_df_nba['team'].str.replace("*","")
base_df_nba['team'] = base_df_nba['team'].str.replace(r"\(.*\)","")

base_df_nba['W/L%'] = base_df_nba['W/L%'].astype(float)
base_df_nba.rename(columns={"W/L%": "Win_Loss_Ratio"}, inplace = True)
base_df_nba.set_index('team', inplace = True)
cols = [0,1,3,4,5,6,7,8]
base_df_nba.drop(base_df_nba.columns[cols],axis=1,inplace=True)
win_loss_dict = base_df_nba.groupby('team').apply(lambda x: x.Win_Loss_Ratio.tolist()).to_dict()



#####get average win/loss for New York & Los Angeles since multiple teams
base_df_nba_filtered_newyork = base_df_nba.reset_index()
base_df_nba_filtered_newyork = base_df_nba_filtered_newyork.drop([0,1,2,3,4,5,6,7,8,9,10,11,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29])
average_newyork = base_df_nba_filtered_newyork['Win_Loss_Ratio'].mean()

base_df_nba_filtered_LA = base_df_nba.reset_index()
base_df_nba_filtered_LA = base_df_nba_filtered_LA.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,28])
average_LA = base_df_nba_filtered_LA['Win_Loss_Ratio'].mean()


In [10]:
#####Question 2 clean cities dataframe NBA and add win/loss ratio using mapping dictionary
cities_nba=pd.read_html("assets/wikipedia_data.html",  na_values = '—')[1]
cities_nba=cities_nba.iloc[:-1,[0,3,5,6,7,8]]
cities_nba['NBA'] = cities_nba['NBA'].str.replace(r"\[.*\]","")



#####Only includes metropolitan areas that did not have NA values for NBA column, taken directly from html file
#####Did not add keys for teams in metro areas with multiple teams in order to make mapping more intuitive. Still averaging values of all teams and inserting in relevant row on later line.
NBAMetropolitianAreaDict  = {'New York City':'New York Knicks\xa0', 
                             'Los Angeles':'Los Angeles Lakers\xa0', 
                             'San Francisco Bay Area':'Golden State Warriors\xa0', 
                             'Chicago':'Chicago Bulls\xa0', 
                             'Dallas–Fort Worth':'Dallas Mavericks\xa0', 
                             'Washington, D.C.':'Washington Wizards\xa0', 
                             'Philadelphia':'Philadelphia 76ers\xa0', 
                             'Boston':'Boston Celtics\xa0',
                             'Minneapolis–Saint Paul':'Minnesota Timberwolves\xa0', 
                             'Denver':'Denver Nuggets\xa0', 
                             'Miami–Fort Lauderdale':'Miami Heat\xa0', 
                             'Phoenix':'Phoenix Suns\xa0', 
                             'Detroit':'Detroit Pistons\xa0',
                             'Toronto':'Toronto Raptors\xa0', 
                             'Houston':'Houston Rockets\xa0',
                             'Atlanta': 'Atlanta Hawks\xa0',
                             'Cleveland': 'Cleveland Cavaliers\xa0',
                             'Charlotte':'Charlotte Hornets\xa0',
                             'Indianapolis': 'Indiana Pacers\xa0',
                             'Milwaukee':'Milwaukee Bucks\xa0',
                             'New Orleans':'New Orleans Pelicans\xa0',
                             'Orlando': 'Orlando Magic\xa0',
                             'Portland': 'Portland Trail Blazers\xa0',
                             'Salt Lake City': 'Utah Jazz\xa0',
                             'San Antonio': 'San Antonio Spurs\xa0',
                             'Sacramento': 'Sacramento Kings\xa0',
                             'Oklahoma City':'Oklahoma City Thunder\xa0',
                             'Memphis':'Memphis Grizzlies\xa0'}



cities_nba = cities_nba.drop(columns=['NFL', 'MLB', 'NHL'])
cities_nba = cities_nba.dropna(subset = ['NBA'])
cities_nba = cities_nba.drop(cities_nba[cities_nba['NBA']== ""].index)
cities_nba.set_index('Metropolitan area', inplace = True)
for i in range(len(cities_nba)):
        city = cities_nba.iloc[i].name
        cities_nba.set_value(city, 'team', NBAMetropolitianAreaDict[city])
cities_nba['Win_Loss_Ratio'] = cities_nba.team.map(win_loss_dict)
cities_nba['Win_Loss_Ratio'] = cities_nba['Win_Loss_Ratio'].astype(str)
cities_nba['Win_Loss_Ratio'] = cities_nba['Win_Loss_Ratio'].str.replace('[', '')
cities_nba['Win_Loss_Ratio'] = cities_nba['Win_Loss_Ratio'].str.replace(']', '')
cities_nba['Win_Loss_Ratio'] = cities_nba['Win_Loss_Ratio'].astype(float)
cities_nba['Population (2016 est.)[8]'] = cities_nba['Population (2016 est.)[8]'].astype(int)
cities_nba = cities_nba.dropna(subset=['Win_Loss_Ratio'])



#####Now, replace New York and LA Win/Loss with average for all teams. Pass into New York and LA Rows
cities_nba.iat[0,3] = average_newyork
cities_nba.iat[1,3] = average_LA


#####Lastly, create lists of metropolitan area Pop and Win/Loss Ratio for NHL. Converted to float/int beforehand.
a_list_nba = cities_nba['Population (2016 est.)[8]'].tolist()
b_list_nba = cities_nba['Win_Loss_Ratio'].tolist()





In [11]:
##Question 3 clean MLB dataframe and create mapping dictinary for cites_mlb df. Use only year 2016
mlb_df=pd.read_csv("assets/mlb.csv")
base_df_mlb = mlb_df
base_df_mlb = base_df_mlb.where(base_df_mlb['year']==2016).dropna(subset= ['year'])
base_df_mlb = base_df_mlb[~base_df_mlb['team'].isin(['Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division'])]
base_df_mlb['team'] = base_df_mlb['team'].str.replace("*","")
base_df_mlb['team'] = base_df_mlb['team'].str.replace(r"\(.*\)","")

base_df_mlb['W-L%'] = base_df_mlb['W-L%'].astype(float)
base_df_mlb.rename(columns={"W-L%": "Win_Loss_Ratio"}, inplace = True)
base_df_mlb.set_index('team', inplace = True)
cols = [0,1,3,4,5]
base_df_mlb.drop(base_df_mlb.columns[cols],axis=1,inplace=True)
win_loss_dict = base_df_mlb.groupby('team').apply(lambda x: x.Win_Loss_Ratio.tolist()).to_dict()



#####get average win/loss for New York, Los Angeles, San Francisco, & Chicago since multiple teams
base_df_mlb_filtered_newyork = base_df_mlb.reset_index()
base_df_mlb_filtered_newyork =  base_df_mlb_filtered_newyork[(base_df_mlb_filtered_newyork['team']=='New York Yankees') | (base_df_mlb_filtered_newyork['team']=='New York Mets')]
average_newyork = base_df_mlb_filtered_newyork['Win_Loss_Ratio'].mean()

base_df_mlb_filtered_LA = base_df_mlb.reset_index()
base_df_mlb_filtered_LA =  base_df_mlb_filtered_LA[(base_df_mlb_filtered_LA['team']=='Los Angeles Dodgers') | (base_df_mlb_filtered_LA['team']=='Los Angeles Angels')]
average_LA = base_df_mlb_filtered_LA['Win_Loss_Ratio'].mean()

base_df_mlb_filtered_SF = base_df_mlb.reset_index()
base_df_mlb_filtered_SF =  base_df_mlb_filtered_SF[(base_df_mlb_filtered_SF['team']=='San Francisco Giants') | (base_df_mlb_filtered_SF['team']=='Oakland Athletics')]
average_SF = base_df_mlb_filtered_SF['Win_Loss_Ratio'].mean()

base_df_mlb_filtered_CH = base_df_mlb.reset_index()
base_df_mlb_filtered_CH =  base_df_mlb_filtered_CH[(base_df_mlb_filtered_CH['team']=='Chicago Cubs') | (base_df_mlb_filtered_CH['team']=='Chicago White Sox')]
average_CH = base_df_mlb_filtered_CH['Win_Loss_Ratio'].mean()

In [12]:
#####Question 3 clean cities dataframe MLB and add win/loss ratio using mapping dictionary
cities_mlb=pd.read_html("assets/wikipedia_data.html",  na_values = '—')[1]
cities_mlb=cities_mlb.iloc[:-1,[0,3,5,6,7,8]]
cities_mlb['MLB'] = cities_mlb['MLB'].str.replace(r"\[.*\]","")



#####Only includes metropolitan areas that did not have NA values for MLB column, taken directly from html file
#####Did not add keys for teams in metro areas with multiple teams in order to make mapping more intuitive. Still averaging values of all teams and inserting in relevant row on later line.
MLBMetropolitianAreaDict  = {'New York City':'New York Yankees', 
                             'Los Angeles':'Los Angeles Dodgers', 
                             'San Francisco Bay Area':'San Francisco Giants', 
                             'Chicago':'Chicago Cubs', 
                             'Dallas–Fort Worth':'Texas Rangers', 
                             'Washington, D.C.':'Washington Nationals', 
                             'Philadelphia':'Philadelphia Phillies', 
                             'Boston':'Boston Red Sox',
                             'Minneapolis–Saint Paul':'Minnesota Twins', 
                             'Denver':'Colorado Rockies', 
                             'Miami–Fort Lauderdale':'Miami Marlins', 
                             'Phoenix':'Arizona Diamondbacks', 
                             'Detroit':'Tigers',
                             'Toronto':'Toronto Blue Jays', 
                             'Houston':'Houston Astros',
                             'Atlanta': 'Atlanta Braves',
                             'Tampa Bay Area': 'Tampa Bay Rays',
                             'Pittsburgh': 'Pittsburgh Pirates',
                             'Cleveland': 'Cleveland Indians',
                             'Seattle': 'Seattle Mariners',
                             'Cincinnati': 'Cincinnati Reds',
                             'Kansas City': 'Kansas City Royals',
                             'St. Louis': 'St. Louis Cardinals',
                             'Baltimore': 'Baltimore Orioles',
                             'Milwaukee':'Milwaukee Brewers',
                             'San Diego':'San Diego Padres'}



cities_mlb = cities_mlb.drop(columns=['NFL', 'NBA', 'NHL'])
cities_mlb = cities_mlb.dropna(subset = ['MLB'])
cities_mlb = cities_mlb.drop(cities_mlb[cities_mlb['MLB']== ""].index)
cities_mlb.set_index('Metropolitan area', inplace = True)
for i in range(len(cities_mlb)):
        city = cities_mlb.iloc[i].name
        cities_mlb.set_value(city, 'team', MLBMetropolitianAreaDict[city])
cities_mlb['Win_Loss_Ratio'] = cities_mlb.team.map(win_loss_dict)
cities_mlb['Win_Loss_Ratio'] = cities_mlb['Win_Loss_Ratio'].astype(str)
cities_mlb['Win_Loss_Ratio'] = cities_mlb['Win_Loss_Ratio'].str.replace('[', '')
cities_mlb['Win_Loss_Ratio'] = cities_mlb['Win_Loss_Ratio'].str.replace(']', '')
cities_mlb['Win_Loss_Ratio'] = cities_mlb['Win_Loss_Ratio'].astype(float)
cities_mlb['Population (2016 est.)[8]'] = cities_mlb['Population (2016 est.)[8]'].astype(int)
cities_mlb = cities_mlb.dropna(subset=['Win_Loss_Ratio'])



#####Now, replace New York and LA Win/Loss with average for all teams. Pass into New York and LA Rows
cities_mlb.iat[0,3] = average_newyork
cities_mlb.iat[1,3] = average_LA
cities_mlb.iat[2,3] = average_SF
cities_mlb.iat[3,3] = average_CH

#####Lastly, create lists of metropolitan area Pop and Win/Loss Ratio for NHL. Converted to float/int beforehand.
a_list_mlb = cities_mlb['Population (2016 est.)[8]'].tolist()
b_list_mlb = cities_mlb['Win_Loss_Ratio'].tolist()



In [13]:
##Question 4 clean NFL dataframe and create mapping dictinary for cites_nfl df. Use only year 2016
nfl_df=pd.read_csv("assets/nfl.csv")
base_df_nfl = nfl_df
base_df_nfl = base_df_nfl.where(base_df_nfl['year']==2016).dropna(subset= ['year'])
base_df_nfl= base_df_nfl[~base_df_nfl['team'].isin(['Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division', 'AFC East', 'AFC North', 'AFC South', 'AFC West', 'NFC East', 'NFC West', 'NFC North', 'NFC South'])]
base_df_nfl['team'] = base_df_nfl['team'].str.replace("*","")
base_df_nfl['team'] = base_df_nfl['team'].str.replace("+","")
base_df_nfl['team'] = base_df_nfl['team'].str.replace(r"\(.*\)","")

base_df_nfl['W-L%'] = base_df_nfl['W-L%'].astype(float)
base_df_nfl.rename(columns={"W-L%": "Win_Loss_Ratio"}, inplace = True)
base_df_nfl.set_index('team', inplace = True)
cols = [0,1,2,3,4,5,6,7,8,9,10,11,13]
base_df_nfl.drop(base_df_nfl.columns[cols],axis=1,inplace=True)
win_loss_dict = base_df_nfl.groupby('team').apply(lambda x: x.Win_Loss_Ratio.tolist()).to_dict()



#####get average win/loss for New York & San Francisco since multiple teams.
#####while LA does have two teams, did not perform this operation for that area since only one LA team has observation in year 2016
base_df_nfl_filtered_newyork = base_df_nfl.reset_index()
base_df_nfl_filtered_newyork =  base_df_nfl_filtered_newyork[(base_df_nfl_filtered_newyork['team']=='New York Giants') | (base_df_nfl_filtered_newyork['team']=='New York Jets')]
average_newyork = base_df_nfl_filtered_newyork['Win_Loss_Ratio'].mean()


base_df_nfl_filtered_SF = base_df_nfl.reset_index()
base_df_nfl_filtered_SF =  base_df_nfl_filtered_SF[(base_df_nfl_filtered_SF['team']=='San Francisco 49ers') | (base_df_nfl_filtered_SF['team']=='Oakland Raiders')]
average_SF = base_df_nfl_filtered_SF['Win_Loss_Ratio'].mean()




In [14]:
#####Question 4 clean cities dataframe MLB and add win/loss ratio using mapping dictionary
cities_nfl=pd.read_html("assets/wikipedia_data.html",  na_values = '—')[1]
cities_nfl=cities_nfl.iloc[:-1,[0,3,5,6,7,8]]
cities_nfl['NFL'] = cities_nfl['NFL'].str.replace(r"\[.*\]","")



#####Only includes metropolitan areas that did not have NA values for MLB column, taken directly from html file
#####Did not add keys for teams in metro areas with multiple teams in order to make mapping more intuitive. Still averaging values of all teams and inserting in relevant row on later line.
NFLMetropolitianAreaDict  = {'New York City':'New York Giants', 
                             'Los Angeles':'Los Angeles Rams', 
                             'San Francisco Bay Area':'San Francisco 49ers', 
                             'Chicago':'Chicago Bears', 
                             'Dallas–Fort Worth':'Dallas Cowboys', 
                             'Washington, D.C.':'Washington Redskins', 
                             'Philadelphia':'Philadelphia Eagles', 
                             'Boston':'New England Patriots',
                             'Minneapolis–Saint Paul':'Minnesota Vikings', 
                             'Denver':'Denver Broncos', 
                             'Miami–Fort Lauderdale':'Miami Dolphins', 
                             'Phoenix':'Arizona Cardinals', 
                             'Detroit':'Lions',
                             'Houston':'Houston Texans',
                             'Atlanta': 'Atlanta Falcons',
                             'Tampa Bay Area': 'Tampa Bay Buccaneers',
                             'Pittsburgh': 'Pittsburgh Steelers',
                             'Cleveland': 'Cleveland Browns',
                             'Seattle': 'Seattle Seahawks',
                             'Cincinnati': 'Cincinnati Bengals',
                             'Kansas City': 'Kansas City Chiefs',
                             'Baltimore': 'Baltimore Ravens',
                             'Charlotte': 'Carolina Panthers',
                             'Indianapolis':'Indianapolis Colts',
                             'Nashville':'Tennessee Titans',
                             'New Orleans': 'New Orleans Saints',
                             'Buffalo': 'Buffalo Bills',
                             'Jacksonville': 'Jacksonville Jaguars',
                             'Green Bay': 'Green Bay Packers'}



cities_nfl = cities_nfl.drop(columns=['MLB', 'NBA', 'NHL'])
cities_nfl = cities_nfl.dropna(subset = ['NFL'])
cities_nfl = cities_nfl.drop(cities_nfl[cities_nfl['NFL']== ""].index)
cities_nfl = cities_nfl[cities_nfl['Metropolitan area'] != 'Toronto']
cities_nfl.set_index('Metropolitan area', inplace = True)
for i in range(len(cities_nfl)):
        city = cities_nfl.iloc[i].name
        cities_nfl.set_value(city, 'team', NFLMetropolitianAreaDict[city])
cities_nfl['Win_Loss_Ratio'] = cities_nfl.team.map(win_loss_dict)
cities_nfl['Win_Loss_Ratio'] = cities_nfl['Win_Loss_Ratio'].astype(str)
cities_nfl['Win_Loss_Ratio'] = cities_nfl['Win_Loss_Ratio'].str.replace('[', '')
cities_nfl['Win_Loss_Ratio'] = cities_nfl['Win_Loss_Ratio'].str.replace(']', '')
cities_nfl['Win_Loss_Ratio'] = cities_nfl['Win_Loss_Ratio'].astype(float)
cities_nfl['Population (2016 est.)[8]'] = cities_nfl['Population (2016 est.)[8]'].astype(int)
cities_nfl = cities_nfl.dropna(subset=['Win_Loss_Ratio'])



#####Now, replace New York and LA Win/Loss with average for all teams. Pass into New York and LA Rows
cities_nfl.iat[0,3] = average_newyork
cities_nfl.iat[3,3] = average_SF


#####Lastly, create lists of metropolitan area Pop and Win/Loss Ratio for NHL. Converted to float/int beforehand.
a_list_nfl = cities_mlb['Population (2016 est.)[8]'].tolist()
b_list_nfl = cities_mlb['Win_Loss_Ratio'].tolist()




In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): # worth 20% of the assignment grade
    population_by_region = a_list_nhl
    win_loss_by_region = b_list_nhl
    return stats.pearsonr(population_by_region, win_loss_by_region)

def nba_correlation(): # worth 20% of the assignment grade
    population_by_region = a_list_nba
    win_loss_by_region = b_list_nba
    return stats.pearsonr(population_by_region, win_loss_by_region)

def mlb_correlation(): # worth 20% of the assignment grade
    population_by_region = a_list_mlb 
    win_loss_by_region = b_list_mlb
    return stats.pearsonr(population_by_region, win_loss_by_region)

def nfl_correlation(): # worth 20% of the assignment grade
    population_by_region = a_list_nfl
    win_loss_by_region = b_list_nfl
    return stats.pearsonr(population_by_region, win_loss_by_region)

def sports_team_performance(): # worth 20% of the assignment grade
    cities_q5_nhl = cities_nhl.reset_index()
    cities_q5_nba = cities_nba.reset_index()
    cities_q5_mlb = cities_mlb.reset_index()
    cities_q5_nfl = cities_nfl.reset_index()
    cities_q5 = cities_nhl.reset_index()
    cities_q5.rename(columns={"Win_Loss_Ratio": "Win_Loss_Ratio_NHL"}, inplace = True)
    cities_q5 =  cities_q5[( cities_q5['Metropolitan area']=='New York City') | ( cities_q5['Metropolitan area']== 'Los Angeles') | ( cities_q5['Metropolitan area'] == 'San Francisco Bay Area')| ( cities_q5['Metropolitan area'] == 'Chicago')]
    merged_df_step1 = pd.merge(cities_q5, cities_q5_nba, on='Metropolitan area', how='inner')
    merged_df_step1.rename(columns={"Win_Loss_Ratio": "Win_Loss_Ratio_NBA"}, inplace = True)
    merged_df_step2 = pd.merge(merged_df_step1, cities_q5_mlb, on='Metropolitan area', how='inner')
    merged_df_step2.rename(columns={"Win_Loss_Ratio": "Win_Loss_Ratio_MLB"}, inplace = True)
    merged_df_final = pd.merge(merged_df_step2, cities_q5_nfl, on='Metropolitan area', how='inner')
    merged_df_final.rename(columns={"Win_Loss_Ratio": "Win_Loss_Ratio_NFL"}, inplace = True)
    cols = [1,2,3,5,6,7,9,10,11,13,14,15]
    merged_df_final.drop(merged_df_final.columns[cols],axis=1,inplace=True)

    merged_df_final = merged_df_final.dropna()._get_numeric_data()
    merged_df_finalcols = pd.DataFrame(columns=merged_df_final.columns)
    p_values = merged_df_finalcols.transpose().join(merged_df_finalcols, how='outer')
    for r in merged_df_final.columns:
        for c in merged_df_final.columns:
            p_values[r][c] = round(stats.ttest_rel(merged_df_final[r], merged_df_final[c])[1], 4)
    return p_values
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"]
    # and df.loc["NFL","NFL"] should return np.nan
 #   p_values=pd.DataFrame({k:np.nan for (k) in ["NFL","NBA","NHL","MLB"]},index=sports)
    #return p_values

In [16]:
print(nhl_correlation())
print(nba_correlation())
print(mlb_correlation())
print(nfl_correlation())

(0.05295640433376599, 0.7930648837387826)
(-0.1876923844402299, 0.33885480510842475)
(0.25509288146039283, 0.21845363299814233)
(0.25509288146039283, 0.21845363299814233)


Conclusion:
--For each correlation calculation, assume an arbitrary significance level of 0.05. 
--For each correlation calculation, assume the null hypothesis is: There is no statistically significant relationship between
  metro area population and win/loss ratio
--For each correlation calculation, assume the alternative hypothesis is: There is a statistically significant relationship between metro area population and win/loss ratio

NHL Correlation: The correlation coefficient is positive but very weak, at only 5%. Furthermore, the p-value is not significant, even at an alpha of 50%. Do not choose to reject the null hypothesis in favor of the alternative.

NBA Correlation: The correlation coefficient is negative but week, and the p-value is only significant at an alpha of
50%. Do not choose to reject the null hypothesis in favor of the alternative

MLB Correlation: The correlation coefficient is modestly positive, at 25%. However, the p-value is only significant at an alpha of 25% and above. At our alpha level, we do not choose to reject the null hypothesis in favor of the alternative.

NFL Correlation: The correlation coefficient is modestly positive, at 25%. However, the p-value is only significant at an alpha of 25% an above. At our alpha level, we do not choose to reject the null hypothesis in favor of the alternative.



In [17]:
sports_team_performance()

,Win_Loss_Ratio_NHL,Win_Loss_Ratio_NBA,Win_Loss_Ratio_MLB,Win_Loss_Ratio_NFL
Win_Loss_Ratio_NHL,NaN,0.5825,0.0439,0.0576
Win_Loss_Ratio_NBA,0.5825,NaN,0.902,0.3706
Win_Loss_Ratio_MLB,0.0439,0.902,NaN,0.078
Win_Loss_Ratio_NFL,0.0576,0.3706,0.078,NaN


Conclusion: 
For each two tailed t-test,  assume the null hypothesis is H0: μd = 0
For each two tailed t-test, assume the alternative hypothesis is H1: μd ≠ 0    (two-tailed)
For each two tailed t-test, assume an arbitrary significance level of 0.05

Upon viewing the output, the only test that has a p-value below our alpha are between (Win_Loss_Ratio_MLB and 	Win_Loss_Ratio_NHL). For this pairing, we can reject the null hypothesis in favor of the alternative hypothesis, meaning the true mean difference between the samples is not zero. In other words, the performance between the teams is not the same. Note that there is one test which produced a p-value that is almost borderline significant, namely between (Win_Loss_Ratio_NFL and Win_Loss_Ratio_NHL).
